In [1]:
import nn_module as n
import math
import torch
from torch import Tensor

In [2]:
def generate_disc_set(nb):
    input = Tensor(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).view(nb,1)
    return input, target

In [3]:
train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

In [4]:
%%capture
mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)


In [5]:
shallow_model = n.networks.sequential([
    n.layers.fully_connected(2,128),
    n.activations.relu(),
    n.layers.fully_connected(128,1)
])

In [6]:
lossf = n.loss.MSE()
optim = n.optimizers.SGD(shallow_model, lossf)

In [7]:
lossf.loss(shallow_model.forward(train_input), train_target)

tensor(238.4619)

In [8]:
optim.train(train_input, train_target, 100, 1e-5)

Epoch 0...
tensor(238.4619)
Epoch 1...
tensor(7.2728)
Epoch 2...
tensor(1.7263)
Epoch 3...
tensor(1.4470)
Epoch 4...
tensor(1.3660)
Epoch 5...
tensor(1.3065)
Epoch 6...
tensor(1.2595)
Epoch 7...
tensor(1.2176)
Epoch 8...
tensor(1.1805)
Epoch 9...
tensor(1.1472)
Epoch 10...
tensor(1.1147)
Epoch 11...
tensor(1.0829)
Epoch 12...
tensor(1.0533)
Epoch 13...
tensor(1.0255)
Epoch 14...
tensor(0.9988)
Epoch 15...
tensor(0.9734)
Epoch 16...
tensor(0.9527)
Epoch 17...
tensor(0.9251)
Epoch 18...
tensor(0.9021)
Epoch 19...
tensor(0.8808)
Epoch 20...
tensor(0.8606)
Epoch 21...
tensor(0.8398)
Epoch 22...
tensor(0.8200)
Epoch 23...
tensor(0.8008)
Epoch 24...
tensor(0.7832)
Epoch 25...
tensor(0.7654)
Epoch 26...
tensor(0.7480)
Epoch 27...
tensor(0.7319)
Epoch 28...
tensor(0.7156)
Epoch 29...
tensor(0.7007)
Epoch 30...
tensor(0.6858)
Epoch 31...
tensor(0.6720)
Epoch 32...
tensor(0.6579)
Epoch 33...
tensor(0.6443)
Epoch 34...
tensor(0.6318)
Epoch 35...
tensor(0.6192)
Epoch 36...
tensor(0.6071)
Epoch 37.

In [9]:
lossf.loss(shallow_model.forward(train_input), train_target)

tensor(0.2617)

In [10]:
lossf.loss(shallow_model.forward(test_input), test_target)

tensor(0.2941)